# PII Analysis with Original Text Column

This notebook runs a Python script to analyze text from a CSV file. It identifies and separates Personally Identifiable Information (PII) from valuable business insights using the Gemini API.

**Change:** This version now includes the original review text as the first column in the final output file.

## Step 1: Install Dependencies

In [ ]:
!pip install -q google-generativeai pandas

## Step 2: Configure API Key

Use the Colab Secrets Manager to securely store your API key.

1.  Click the **key icon (🔑)** in the left-hand sidebar.
2.  Create a new secret named `GEMINI_API_KEY`.
3.  Paste your API key into the **Value** field.

In [ ]:
import os
from google.colab import userdata

try:
    os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
    print("API Key configured successfully.")
except userdata.SecretNotFoundError:
    print('Secret not found. Please follow the instructions in Step 2 to set up your API key.')
    exit()

## Step 3: Upload Your CSV File

Run the following cell and upload your `yelp_200.csv` file.

In [ ]:
from google.colab import files

print("Please upload your 'yelp_200.csv' file.")
uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

## Step 4: Define the Analysis Functions

This cell contains the complete analysis script, including all prompts and functions.

In [ ]:
import google.generativeai as genai
import pandas as pd
import json
import io
import os
import time

# --- Configuration ---
# The API key is loaded from the environment variable set in Step 2.
try:
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
except KeyError:
    print("GEMINI_API_KEY not found in environment variables. Please complete Step 2.")
    exit()

# --- The Final, Detailed Prompt ---
SYSTEM_PROMPT = """
You are an AI assistant specializing in data analysis with a focus on privacy and utility. Your task is to analyze text data, identify and separate Personally Identifiable Information (PII) from valuable business insights.
"""

USER_PROMPT_TEMPLATE = """
Analyze the following text and provide the output as a single, minified JSON object.

Definitions:

* **Privacy Aspect**: Any information that could be used to identify an individual. **It is critical to retrieve all PII completely. Failure to extract every instance can lead to privacy leakage.** For example, ensure all names and all email addresses in a text are listed.
    * `Name`: Full names or specific usernames.
    * `Location/Address`: Specific street addresses, or precise locations.
    * `Age`: Specific ages.
    * `Gender`: Gender identification.
    * `Date of Birth`: Specific dates of birth.
    * `Date of Event`: Specific dates of events.
    * `SSN`: Social Security Numbers.
    * `Email Address`: Email addresses.
    * `Social Media Account`: Social media handles, but only if a specific username (e.g., @username) is provided.
    * `Employment Details`: Specific job titles or employers.
    * `Miscellaneous`: Any other data that could be considered PII or sensitive but does not fit the categories above (e.g., unique identifiers, internal folder paths, specific phone numbers). If none, use "N/A".
    * If no PII is found in a category, use "N/A".
* **Utility Aspect**: Information that provides valuable insights for business intelligence. This includes:
    * `Category`: The general category of the business or service (e.g., "Restaurant", "Healthcare", "Retail", "Corporate Communication").
    * `Sentiment`: The overall sentiment of the review, categorized as **Positive**, **Negative**, **Neutral**, **Mixed** (contains both positive and negative elements), or **Informational**.
    * `Keywords`: A comma-separated list of the most important words. If an age is mentioned in a medical context, include the general age range (e.g., 'late 60s') as a keyword.
    * `Summary`: A short, one-sentence summary. This summary must not contain any private information. If an age is mentioned in a medical context, refer to it as a general range (e.g., 'a patient in their late 60s').

Output Format:
The output must be a single, minified JSON object with the following keys: "Name", "Location/Address", "Age", "Gender", "Date of Birth", "Date of Event", "SSN", "Email Address", "Social Media Account", "Employment Details", "Miscellaneous", "Category", "Sentiment", "Keywords", "Summary"

---
### **Examples:**

**Example 1 Input Text**: "This place is amazing! The service was top-notch and the food was delicious. I'd give it a 10 out of 10, will definitely be back. My server was David."
**Example 1 Output JSON**: {{"Name": "David", "Location/Address": "N/A", "Age": "N/A", "Gender": "N/A", "Date of Birth": "N/A", "Date of Event": "N/A", "SSN": "N/A", "Email Address": "N/A", "Social Media Account": "N/A", "Employment Details": "server", "Miscellaneous": "N/A", "Category": "Restaurant", "Sentiment": "Positive", "Keywords": "amazing, service, top-notch, food, delicious", "Summary": "A customer had an amazing experience, praising the delicious food and top-notch service."}}

---
**Example 2 Input Text**: "Patient John Smith, a 68-year-old male (DOB: 1957-04-12), was admitted to University Hospital on September 15, 2025..."
**Example 2 Output JSON**: {{"Name": "John Smith, Dr. Emily Carter", "Location/Address": "University Hospital", "Age": 68, "Gender": "male", "Date of Birth": "1957-04-12", "Date of Event": "September 15, 2025; September 22, 2025", "SSN": "N/A", "Email Address": "N/A", "Social Media Account": "N/A", "Employment Details": "retired librarian", "Miscellaneous": "N/A", "Category": "Healthcare", "Sentiment": "Informational", "Keywords": "pneumonia, chest X-ray, bilateral infiltrates, Levofloxacin, Type 2 Diabetes, Metformin, Augmentin, discharge, follow-up, late 60s", "Summary": "A male patient in his late 60s with a history of diabetes was successfully treated for severe pneumonia with intravenous antibiotics and discharged with a prescription for oral antibiotics and instructions for follow-up care."}}
---

**Analyze this data:**
{text_to_analyze}
"""

def analyze_single_text(text_to_analyze):
    """
    Analyzes a single piece of text using the Gemini API and returns the result as a dictionary.
    """
    print(f"  > Analyzing text...")

    model = genai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=SYSTEM_PROMPT
    )

    full_prompt = USER_PROMPT_TEMPLATE.format(text_to_analyze=text_to_analyze)

    try:
        response = model.generate_content(full_prompt)
        response_text = response.text.strip()

        if response_text.startswith("```json"):
            response_text = response_text[7:-3].strip()

        print("  > Analysis received.")
        json_object = json.loads(response_text)
        return json_object

    except json.JSONDecodeError:
        print("  > FAILED to decode JSON from model response:")
        print(f"  > Response: {response_text}")
        return None
    except Exception as e:
        print(f"  > An error occurred during API call or parsing: {e}")
        return None

def process_csv_and_save_results(input_csv_path, output_json_path, output_csv_path, text_column='Reviews', rating_column='Rating'):
    """
    Reads a CSV, processes each row, saves the complete analysis to a JSON file,
    and then converts that JSON file to a final CSV.
    """
    print(f"Starting to process '{input_csv_path}'...")

    try:
        df = pd.read_csv(input_csv_path)
    except FileNotFoundError:
        print(f"Error: The file '{input_csv_path}' was not found.")
        return
    except KeyError as e:
        print(f"Error: Missing required column in the input CSV. Please ensure you have '{text_column}' and '{rating_column}' columns. Details: {e}")
        return

    all_results = []

    for index, row in df.iterrows():
        print(f"\nProcessing row {index}...")
        text_to_process = str(row[text_column])
        input_rating = row[rating_column]

        analysis_result = analyze_single_text(text_to_process)

        if analysis_result:
            # Add the original review text to the results dictionary.
            # This happens AFTER the API call, so it costs no tokens.
            analysis_result['Original Text'] = text_to_process

            # Add the rating from the original file
            analysis_result['Rating'] = input_rating

            all_results.append(analysis_result)
            print(f"  > Stored result for row {index}. Using provided rating: {input_rating}")
        else:
            print(f"  > Skipped row {index} due to an error.")

        time.sleep(2)

    print("\n-------------------------------------")
    print(f"Analysis complete for all rows.")

    try:
        with open(output_json_path, 'w') as f:
            json.dump(all_results, f, indent=4)
        print(f"Successfully saved all results to '{output_json_path}'")
    except Exception as e:
        print(f"Error saving to JSON file: {e}")
        return

    try:
        final_df = pd.DataFrame(all_results)

        # Added "Original Text" to the beginning of the column order.
        column_order = [
            "Original Text", "Name", "Location/Address", "Age", "Gender", "Date of Birth", "Date of Event",
            "SSN", "Email Address", "Social Media Account", "Employment Details", "Miscellaneous",
            "Category", "Sentiment", "Rating", "Keywords", "Summary"
        ]

        # Reorder the dataframe to match the desired column order
        final_df = final_df[column_order]
        final_df.to_csv(output_csv_path, index=False)
        print(f"Successfully converted JSON to '{output_csv_path}'")
    except Exception as e:
        print(f"Error converting JSON to CSV: {e}")

## Step 5: Run the Analysis

The final cell executes the script. It defines the input and output filenames and calls the main processing function.

In [ ]:
INPUT_CSV = "yelp_200.csv"
OUTPUT_JSON = "analysis_results200.json"
OUTPUT_CSV = "analysis_results200.csv"

TEXT_COLUMN_NAME = "Reviews"
RATING_COLUMN_NAME = "Rating"

process_csv_and_save_results(
    input_csv_path=INPUT_CSV,
    output_json_path=OUTPUT_JSON,
    output_csv_path=OUTPUT_CSV,
    text_column=TEXT_COLUMN_NAME,
    rating_column=RATING_COLUMN_NAME
)

## Step 6: Verify and Download Output

After the script finishes, run the cell below to list the files in the current directory. You should see `analysis_results200.json` and `analysis_results200.csv`. You can download them from the file browser on the left.

In [ ]:
!ls -l